In [1]:
import sys
sys.path.append("..")
import random

In [2]:
import dataset.ade as ade
from dataset.ade import AdeSegmentationIncremental
import dataset.transform as transforms
import torchvision as tv
import numpy as np
import os

In [3]:
data = ade.AdeSegmentation('/vandal/datasets', train=True)


In [4]:
def group_images(dataset, labels):
    # Group images based on the label in LABELS
    idxs = [[] for lab in labels]

    labels_cum = labels + [0, 255]
    for i in range(len(dataset)):
        cls = np.unique(np.array(dataset[i][1]))
        if all(x in labels_cum for x in cls):
            for x in cls:
                if x in labels:
                    idxs[x].append(i)
        else:
            print(f"Not all in labels_cum: {cls}")
        if i % 1000 == 0:
            print(f"Done {i} / {len(dataset)}")
    return idxs

idxs = group_images(data, list(range(151)))
# imgs_to_cls map all the images to all the classes contained

Done 0 / 20210
Done 1000 / 20210
Done 2000 / 20210
Done 3000 / 20210
Done 4000 / 20210
Done 5000 / 20210
Done 6000 / 20210
Done 7000 / 20210
Done 8000 / 20210
Done 9000 / 20210
Done 10000 / 20210
Done 11000 / 20210
Done 12000 / 20210
Done 13000 / 20210
Done 14000 / 20210
Done 15000 / 20210
Done 16000 / 20210
Done 17000 / 20210
Done 18000 / 20210
Done 19000 / 20210
Done 20000 / 20210


In [5]:
len_idxs = [len(idxs[i]) for i in range(151)]
print(len_idxs)

[20208, 11588, 6042, 8240, 9314, 6674, 6579, 3990, 1825, 4687, 2421, 2851, 3061, 5069, 1791, 3276, 4266, 1690, 3995, 2148, 3258, 3161, 702, 3295, 1191, 1404, 667, 650, 1229, 1132, 479, 1172, 521, 1385, 698, 781, 378, 3083, 404, 803, 1447, 411, 1437, 799, 2646, 549, 367, 310, 1181, 287, 468, 402, 130, 561, 880, 84, 172, 198, 930, 139, 564, 320, 261, 275, 335, 792, 395, 1308, 1112, 266, 658, 330, 531, 369, 144, 265, 324, 304, 170, 68, 65, 247, 492, 2508, 440, 135, 583, 532, 1987, 71, 618, 133, 83, 178, 992, 181, 451, 41, 307, 965, 117, 354, 108, 556, 52, 99, 57, 66, 77, 340, 66, 465, 50, 622, 80, 59, 530, 282, 73, 213, 136, 116, 266, 58, 417, 319, 1191, 97, 347, 52, 246, 108, 201, 284, 268, 1020, 1281, 528, 452, 671, 397, 92, 228, 556, 217, 206, 126, 178, 504, 773, 421]


In [6]:
class_members = {}
for i,l in enumerate(idxs):
    if i==0:
        continue
    class_members[i]=len(l)

#random.seed(42)
a = np.arange(151) 
a=a[1:]
a=np.random.permutation(a)
ordered_classes = class_members.items()

# map the image to the class contained, the inverse of idxs.
imgs_to_cls = {}
for j,v in ordered_classes:
        for i in list(idxs[j]):
            if not i in imgs_to_cls.keys():
                imgs_to_cls[i]=[]
            imgs_to_cls[i].append(j)
print(len(imgs_to_cls))

20196


In [7]:
# check if all image has at least 1 object labeled (not all bkg)
for i in imgs_to_cls.keys():
    if len(imgs_to_cls[i])<1:
        print(i)

In [8]:
# make the order for the dataset!
# map_class_to_set maps the class to the corresponding step

base_len = 50 # first training step: # of images
inc_len = 50  # following training step (assume same dimension)
sets = 2      # Num of incremental steps. E.g. the 50-50-50 has 2 inc. steps

order = np.arange(1,151)
np.random.shuffle(order) # make a random order

map_set_to_class = {}
map_class_to_set = {}

map_set_to_class[0] = order[0:base_len]
map_class_to_set.update({x:0 for x in map_set_to_class[0]})
offset = base_len
for i in range(1, sets+1):
    map_set_to_class[i] = order[offset : offset + inc_len]
    offset = offset + inc_len
    map_class_to_set.update({x:i for x in map_set_to_class[i]})

print({i:sorted(list(x)) for i,x in map_set_to_class.items()})
print({i:len(x) for i,x in map_set_to_class.items()})

map_class_to_set[1]


{0: [3, 5, 6, 13, 14, 20, 23, 25, 27, 31, 35, 39, 43, 44, 46, 47, 49, 50, 51, 52, 54, 56, 57, 58, 63, 65, 68, 69, 79, 86, 88, 90, 95, 98, 105, 108, 109, 113, 114, 118, 120, 125, 126, 131, 132, 136, 138, 146, 149, 150], 1: [7, 8, 16, 17, 18, 19, 21, 28, 29, 30, 34, 36, 38, 61, 66, 67, 71, 73, 75, 76, 78, 81, 82, 83, 84, 85, 87, 89, 91, 94, 97, 99, 103, 104, 106, 107, 115, 117, 122, 124, 127, 128, 133, 134, 140, 141, 144, 145, 147, 148], 2: [1, 2, 4, 9, 10, 11, 12, 15, 22, 24, 26, 32, 33, 37, 40, 41, 42, 45, 48, 53, 55, 59, 60, 62, 64, 70, 72, 74, 77, 80, 92, 93, 96, 100, 101, 102, 110, 111, 112, 116, 119, 121, 123, 129, 130, 135, 137, 139, 142, 143]}
{0: 50, 1: 50, 2: 50}


2

In [9]:
# This is the code who actually make the split.

all_added = set()

imgs={}
others = {}
every=set()
added = [0 for i in range(151)]
for i in range(sets+1):
    others[i]=set()
    imgs[i]=set()

# This is the most imporant function. 
# It computes the score of the image to be assinged to a class
def score(ratio,imgs,expected,minval=False,w1=120,w2=1000,w3=1 ):
    return (1.-ratio)*w1+w3*minval

for i in imgs_to_cls.keys(): # loop for every image in the dataset
    # img_counts = num of images per class for the class in the actual image  
    img_counts = [class_members[j] for j in imgs_to_cls[i]] 
    # ratio = images assigned to the class / total number of images for the class  
    ratios = [(added[j]+0.0)/class_members[j] for j in imgs_to_cls[i]]
    added_counts = [added[j] for j in imgs_to_cls[i]]
    # assignments = set of each class in the image
    assignments = [map_class_to_set[j] for j in imgs_to_cls[i]]
    
    scores = [score(ratios[c],added_counts[c],class_members[j],1./(img_counts[c]/sum(img_counts))) for c,j in enumerate(imgs_to_cls[i])]
    
    # take the highest scorer class
    cl = scores.index(max(scores))
    # take the set of the higher scorer class
    a=assignments[cl]
    
    # add the image to the step images
    imgs[a].update([i])
    for j,ac in enumerate(assignments):
        if ac==a:
            # increment the number of images assigned to the classes in the current image contained in the same step.
            added[imgs_to_cls[i][j]] += 1
            
            
for i in range(1,len(added)):        
        assignment = map_class_to_set[i]
        ratios = [len(set(idxs[i]).intersection(imgs[j]))/class_members[i] for j in range(0,sets+1)]
        if ratios[assignment]<0.5 or (ratios[assignment]<1. and added[i]<100):
            print(i,ratios[assignment], sum(ratios[1:]), class_members[i], len(set(idxs[i]).intersection(imgs[assignment])))
s=0
for i in range(sets+1):
    print(len(imgs[i]))
    s+=len(imgs[i])
    
print(s)
            


1 0.3566620642043493 0.6568864342423196 11588 4133
2 0.29344587884806356 0.6607083747103608 6042 1773
3 0.3963592233009709 0.603640776699029 8240 3266
4 0.35129911960489585 0.6398969293536612 9314 3272
5 0.37413844770752175 0.6258615522924782 6674 2497
6 0.3731570147438821 0.6268429852561179 6579 2455
7 0.43107769423558895 0.7172932330827068 3990 1720
8 0.3254794520547945 0.6202739726027398 1825 594
9 0.3522509067633881 0.6460422445060807 4687 1651
10 0.3985956216439488 0.6600578273440727 2421 965
11 0.3444405471764293 0.6545071904594879 2851 982
12 0.31950343025155176 0.7624959163672003 3061 978
13 0.34819491023870586 0.6518050897612941 5069 1765
14 0.47068676716917923 0.5293132328308208 1791 843
15 0.35836385836385837 0.6504884004884005 3276 1174
16 0.2817627754336615 0.597515236755743 4266 1202
17 0.4325443786982249 0.6230769230769231 1690 731
18 0.3406758448060075 0.672090112640801 3995 1361
19 0.30353817504655495 0.5963687150837989 2148 652
20 0.38275015346838553 0.617249846531614

In [10]:
def class_occurrencies(data, idxs, n_cl=151):
    classes = np.zeros(n_cl)
    for i in idxs:
        for cls in imgs_to_cls[i]:
            classes[cls] += 1
    return classes

# for i in range(sets+1):
#     print(f"{i} : {len(imgs[i])}")
#     for j in range(sets+1):
#         print(f"{i} : {j}={map_set_to_class[j]}-{class_occurrencies(data, imgs[i])[map_set_to_class[j]]}")
#     print("\n")


In [11]:
map_set_to_class_id = {}
for i in range(sets+1):
    map_set_to_class_id[i] = np.array(list(range(151)))[map_set_to_class[i]]
map_set_to_class_id

{0: array([125,   6,  54,  56,  57,  63, 150,  90,  58,  51,  68, 136,  31,
         14,   3, 118,  20,  86,  50,  13, 132,  44,  69,  46,  23, 108,
         43, 138, 126, 149,  88,  27,  47, 109,   5,  65,  49, 105,  25,
        113,  98,  95,  35, 146,  39, 120, 131, 114,  79,  52]),
 1: array([ 28,  30,  99,  83, 128, 148,   7,  16,  29,  75,  34,  71, 144,
         67,  84, 147, 122,  78,  81, 134, 117,  94,  82,  36,  73, 140,
         21,  38,  76, 141, 106, 107, 133,  18,  97,  61, 115, 127,  87,
        124,   8, 145,  66, 103,  85,  19,  91,  89, 104,  17]),
 2: array([ 32,  22,  48, 112,  40,  59,  60,  10, 121, 111,   1, 116, 129,
         37,  53,  70,  77,  74,  24, 100,  33,   9,  12,   2,  11,  80,
         15, 102, 143, 101,  72, 137, 110,  93,  41, 135,  55,  26,  92,
        123,  96, 142,  62,  45,  42, 139, 130,  64, 119,   4])}

In [12]:
path = '../split/'
os.makedirs(path, exist_ok=True)
with open(path + 'order.txt', 'w') as f:
    f.write(str({i:sorted(list(x)) for i,x in map_set_to_class_id.items()}))
for i in range(sets+1):
    np.save(path + 'train-'+str(i)+'.npy',  np.array(list(imgs[i])))
    print(len(imgs[i]))

7646
6149
6401
